In [1]:
!pip install rdkit
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 61.7 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import average_precision_score
import numpy as np
import lightgbm as lgb
import optuna
import duckdb
from sklearn.metrics import roc_auc_score

In [4]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

In [5]:
con = duckdb.connect()

# 각 단백질에 대해 binds=0,1 데이터를 각각 12000개씩 불러오기
df_brd4_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 20000
""").df()

df_brd4_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'BRD4'
    ORDER BY random()
    LIMIT 20000
""").df()

df_hsa_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 20000
""").df()

df_hsa_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'HSA'
    ORDER BY random()
    LIMIT 20000
""").df()

df_seh_0 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 0 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 20000
""").df()

df_seh_1 = con.query(f"""
    SELECT * 
    FROM parquet_scan('{train_path}') 
    WHERE binds = 1 AND protein_name = 'sEH'
    ORDER BY random()
    LIMIT 20000
""").df()

# 데이터프레임 결합
df = pd.concat([df_brd4_0, df_brd4_1, df_hsa_0, df_hsa_1, df_seh_0, df_seh_1], axis=0).reset_index(drop=True)
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
# ECFP 생성 함수
def generate_ecfp(molecule, radius=2, bits=2048):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# MACCS keys 생성 함수
def generate_maccs_keys(molecule):
    if molecule is None:
        return None
    return list(AllChem.GetMACCSKeysFingerprint(molecule))

# 분자의 물리화학적 특성을 추출하는 함수
def generate_physchem_features(molecule):
    if molecule is None:
        return [np.nan] * 6
    return [
        Descriptors.MolWt(molecule),                   # 분자량
        Descriptors.MolLogP(molecule),                 # 로그 P
        Descriptors.NumHDonors(molecule),              # 수소 결합 공여자 수
        Descriptors.NumHAcceptors(molecule),           # 수소 결합 수용체 수
        Descriptors.TPSA(molecule),                    # TPSA (극성 표면적)
        Descriptors.NumRotatableBonds(molecule)        # 회전 가능한 결합 수
    ]

In [7]:
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
df['physchem'] = df['molecule'].apply(generate_physchem_features)
df['ecfp'] = df['molecule'].apply(generate_ecfp)
df['maccs'] = df['molecule'].apply(generate_maccs_keys)

In [8]:
# 물리화학적 특성을 정규화
physchem_features = np.array(df['physchem'].tolist())
scaler = StandardScaler()
physchem_features_scaled = scaler.fit_transform(physchem_features)


In [9]:
# ECFP, MACCS keys, 정규화된 물리화학적 특성을 결합
df['features'] = [ecfp + maccs + physchem_scaled.tolist() for ecfp, maccs, physchem_scaled in zip(df['ecfp'], df['maccs'], physchem_features_scaled)]


In [10]:
# 단백질 이름을 원-핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))


In [11]:
# 최종 입력 데이터 생성
X = np.array([features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())])
y = np.array(df['binds'].tolist())


In [12]:
# 최종 입력 데이터 생성
X = np.array([features + list(protein) for features, protein in zip(df['features'].tolist(), protein_onehot.tolist())])
y = np.array(df['binds'].tolist())


In [13]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna를 사용한 하이퍼파라미터 최적화
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 0.9),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255)
    }
    
    model = lgb.LGBMClassifier(**param)
    
    # fit 메서드에서 early_stopping_rounds와 eval_metric을 명시적으로 지정
    model.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)], 
              eval_metric='auc')
    
    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 최적의 파라미터 출력
print("Best parameters found: ", study.best_params)

[I 2024-06-03 12:50:04,190] A new study created in memory with name: no-name-4d427a49-7dbf-411d-b34c-3fcbcec36337


[LightGBM] [Info] Number of positive: 48149, number of negative: 47851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.013059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4741
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1975
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501552 -> initscore=0.006208
[LightGBM] [Info] Start training from score 0.006208
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-03 12:50:15,567] Trial 0 finished with value: 0.9671749531328786 and parameters: {'n_estimators': 101, 'max_depth': 8, 'learning_rate': 0.2366605912888248, 'subsample': 0.6987756361967712, 'colsample_bytree': 0.7653816648957348, 'num_leaves': 93}. Best is trial 0 with value: 0.9671749531328786.


[LightGBM] [Info] Number of positive: 48149, number of negative: 47851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.421207 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4741
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1975
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501552 -> initscore=0.006208
[LightGBM] [Info] Start training from score 0.006208
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-03 12:50:44,701] Trial 1 finished with value: 0.9720653112637178 and parameters: {'n_estimators': 164, 'max_depth': 11, 'learning_rate': 0.18587475087902058, 'subsample': 0.7131432847962176, 'colsample_bytree': 0.7206836234583778, 'num_leaves': 222}. Best is trial 1 with value: 0.9720653112637178.


...

[LightGBM] [Info] Number of positive: 48149, number of negative: 47851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.156695 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4741
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1975
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501552 -> initscore=0.006208
[LightGBM] [Info] Start training from score 0.006208


[I 2024-06-03 13:27:49,997] Trial 48 finished with value: 0.9617174520080003 and parameters: {'n_estimators': 129, 'max_depth': 10, 'learning_rate': 0.05722706301240299, 'subsample': 0.7124646943947053, 'colsample_bytree': 0.6060205109248582, 'num_leaves': 87}. Best is trial 26 with value: 0.9745242042490176.


[LightGBM] [Info] Number of positive: 48149, number of negative: 47851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.213548 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4741
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1975
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501552 -> initscore=0.006208
[LightGBM] [Info] Start training from score 0.006208
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

[I 2024-06-03 13:28:32,982] Trial 49 finished with value: 0.973323977539065 and parameters: {'n_estimators': 366, 'max_depth': 11, 'learning_rate': 0.09130416127541595, 'subsample': 0.6113842181649033, 'colsample_bytree': 0.660856266774643, 'num_leaves': 101}. Best is trial 26 with value: 0.9745242042490176.


Best parameters found:  {'n_estimators': 465, 'max_depth': 11, 'learning_rate': 0.11115531825960226, 'subsample': 0.6271813693364978, 'colsample_bytree': 0.7586120273366164, 'num_leaves': 102}


In [14]:
# 최적의 파라미터로 모델 학습
best_params = study.best_params
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 48149, number of negative: 47851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.160984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4741
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 1975
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501552 -> initscore=0.006208
[LightGBM] [Info] Start training from score 0.006208
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

LGBMClassifier(colsample_bytree=0.7586120273366164,
               learning_rate=0.11115531825960226, max_depth=11,
               n_estimators=465, num_leaves=102, subsample=0.6271813693364978)

In [15]:
# 테스트 데이터 예측 및 평가
y_pred = best_model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred)
print(f'Test AUC Score: {auc_score:.4f}')

Test AUC Score: 0.9745


In [16]:
# 테스트 데이터 예측 및 저장
test_file = '/kaggle/input/leash-BELKA/test.csv'  
output_file = '/kaggle/working/submission9.csv'  # 출력 파일 경로

In [17]:
import os

In [18]:
for df_test in pd.read_csv(test_file, chunksize=100000):
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['maccs'] = df_test['molecule'].apply(generate_maccs_keys)
    df_test['physchem'] = df_test['molecule'].apply(generate_physchem_features)
    
    # 테스트 데이터의 물리화학적 특성 정규화
    physchem_features_test = np.array(df_test['physchem'].tolist())
    physchem_features_test_scaled = scaler.transform(physchem_features_test)
    
    df_test['features'] = [ecfp + maccs + physchem_scaled.tolist() for ecfp, maccs, physchem_scaled in zip(df_test['ecfp'], df_test['maccs'], physchem_features_test_scaled)]
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    X_test = np.array([features + list(protein) for features, protein in zip(df_test['features'].tolist(), protein_onehot.tolist())])
    
    probabilities = best_model.predict_proba(X_test)[:, 1]
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))